In [2]:
import pandas as pd
import numpy as np

In [3]:
sampel = pd.read_csv('D:/Belajar python/mlc_dataslayer/sample_submission.csv')
test = pd.read_csv('D:/Belajar python/mlc_dataslayer/test.csv')
df = pd.read_csv('D:/Belajar python/mlc_dataslayer/train.csv')

In [4]:
df.head()

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km)
0,1,FOLD,PICKUP TRUCK - STANDARD,3.5,6.0,A6,X,8.64 liters per 100 km,6.14 L/100 km,37.59 mpg Imp.,306.0
1,2,CHEVO,PICKUP TRUCK - STANDARD,5.3,NaN,A6,E,27.27 liters per 100 km,30.76 liters per 100 km,28.84 L/100 km,283.0
2,3,BMV,SUBCOMPACT,4.4,NaN,M6,Z,0.17 L/10km,2.03 liters per 100 km,zero,329.0
3,4,KIO,SUV - SMALL,NaN,4.0,AS6,X,6.52 km per L,1.59 L/10km,NaN,270.0
4,5,BARUSU,MINICOMPACT,3.0,6.0,unspecified,Z,20.15 L/100km,0.60 L/10km,1.38 L/10km,193.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137141 entries, 0 to 137140
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Id                     137141 non-null  int64  
 1   Make                   137141 non-null  object 
 2   Vehicle Class          135816 non-null  object 
 3   Engine Size(L)         120522 non-null  object 
 4   Cylinders              122986 non-null  object 
 5   Transmission           134536 non-null  object 
 6   Fuel Type              135286 non-null  object 
 7   Fuel Consumption City  134719 non-null  object 
 8   Fuel Consumption Hwy   134573 non-null  object 
 9   Fuel Consumption Comb  134559 non-null  object 
 10  CO2 Emissions(g/km)    137141 non-null  float64
dtypes: float64(1), int64(1), object(9)
memory usage: 11.5+ MB


In [6]:
df.isna().sum()

Id                           0
Make                         0
Vehicle Class             1325
Engine Size(L)           16619
Cylinders                14155
Transmission              2605
Fuel Type                 1855
Fuel Consumption City     2422
Fuel Consumption Hwy      2568
Fuel Consumption Comb     2582
CO2 Emissions(g/km)          0
dtype: int64

In [7]:
def remove_units(x):
    to_remove = ['liters per 100 km', 'L/10km', 'km per L', 'L/100km', 'MPG (AS)',
       'mpg Imp.', 'L/100 km', 'km/L']
    for i in to_remove:
        x = str(x).replace(i,'')
    return x

def convert_units(x):
    if x in ['km/L', 'km per L']:
        return 'km/L'
    elif x in ['L/100 km', 'L/100km', 'liters per 100 km']:
        return 'L/100km'
    else:
        return x

def convert_fuel_city(row):
    if row['Fuel Consumption City units'] == 'L/10km':
        return row['Fuel Consumption City'] * 10
    elif row['Fuel Consumption City units'] == 'MPG (AS)':
        return 235.215 / row['Fuel Consumption City']
    elif row['Fuel Consumption City units'] == 'mpg Imp.':
        return 282.481 / row['Fuel Consumption City']
    elif row['Fuel Consumption City units'] == 'km/L':
        return 100 / row['Fuel Consumption City']
    else:
        return row['Fuel Consumption City']
    
def convert_fuel_hwy(row):
    if row['Fuel Consumption Hwy units'] == 'L/10km':
        return row['Fuel Consumption Hwy'] * 10
    elif row['Fuel Consumption Hwy units'] == 'MPG (AS)':
        return 235.215 / row['Fuel Consumption Hwy']
    elif row['Fuel Consumption Hwy units'] == 'mpg Imp.':
        return 282.481 / row['Fuel Consumption Hwy']
    elif row['Fuel Consumption Hwy units'] == 'km/L':
        return 100 / row['Fuel Consumption Hwy']
    else:
        return row['Fuel Consumption Hwy']

def convert_fuel_comb(row):
    if row['Fuel Consumption Comb units'] == 'L/10km':
        return row['Fuel Consumption Comb'] * 10
    elif row['Fuel Consumption Comb units'] == 'MPG (AS)':
        return 235.215 / row['Fuel Consumption Comb']
    elif row['Fuel Consumption Comb units'] == 'mpg Imp.':
        return 282.481 / row['Fuel Consumption Comb']
    elif row['Fuel Consumption Comb units'] == 'km/L':
        return 100 / row['Fuel Consumption Comb']
    else:
        return row['Fuel Consumption Comb']
    
def data_preprocessing(data1):
    data = data1.copy()
    data['Fuel Consumption City'] = data['Fuel Consumption City'].map(lambda x: np.nan if x in ['0','not-available','-9999','-1','zero','9999'] else x)
    data['Fuel Consumption Comb'] = data['Fuel Consumption Comb'].map(lambda x: np.nan if x in ['0','not-available','-9999','-1','zero','9999'] else x)
    data['Fuel Consumption Hwy'] = data['Fuel Consumption Hwy'].map(lambda x: np.nan if x in ['0','not-available','-9999','-1','zero','9999'] else x)

    data['Fuel Consumption City units'] = data['Fuel Consumption City'].map(lambda x: str(x)[5:] if pd.isnull(x) == False else x)
    data['Fuel Consumption Hwy units'] = data['Fuel Consumption Hwy'].map(lambda x: str(x)[5:] if pd.isnull(x) == False else x)
    data['Fuel Consumption Comb units'] = data['Fuel Consumption Comb'].map(lambda x: str(x)[5:] if pd.isnull(x) == False else x)

    # City
    for i in range(7):
        data['Fuel Consumption City units'] = data['Fuel Consumption City units'].map(lambda x: str(x)[1:] if str(x)[0].isalpha() == False else x)

    # HWY
    for i in range(5):
        data['Fuel Consumption Hwy units'] = data['Fuel Consumption Hwy units'].map(lambda x: str(x)[1:] if str(x)[0].isalpha() == False else x)

    # Comb
    for i in range(5):
        data['Fuel Consumption Comb units'] = data['Fuel Consumption Comb units'].map(lambda x: str(x)[1:] if str(x)[0].isalpha() == False else x)

    data['Fuel Consumption City'] = data['Fuel Consumption City'].apply(remove_units).astype(float)
    data['Fuel Consumption Hwy'] = data['Fuel Consumption Hwy'].apply(remove_units).astype(float)
    data['Fuel Consumption Comb'] = data['Fuel Consumption Comb'].apply(remove_units).astype(float)

    data['Fuel Consumption City units'] = data['Fuel Consumption City units'].apply(convert_units)
    data['Fuel Consumption Hwy units'] = data['Fuel Consumption Hwy units'].apply(convert_units)
    data['Fuel Consumption Comb units'] = data['Fuel Consumption Comb units'].apply(convert_units)

    data['Fuel Consumption City'] = data.apply(convert_fuel_city, axis=1)
    data['Fuel Consumption Hwy'] = data.apply(convert_fuel_hwy, axis=1)
    data['Fuel Consumption Comb'] = data.apply(convert_fuel_comb, axis=1)

    data['Transmission'] = data['Transmission'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing','unestablished',
                                                                              'unspecified','na','not-available','-1'] else x)
    data['Cylinders'] = data['Cylinders'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing','unestablished',
                                                                        'unspecified','na','not-available','-1'] else x)
    data['Engine Size(L)'] = data['Engine Size(L)'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing',
                                                                                  'unestablished','unspecified','na','not-available','-1'] else x)
    data['Vehicle Class'] = data['Vehicle Class'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing',
                                                                                'unestablished','unspecified','na','not-available','-1'] else x)
    data['Fuel Type'] = data['Fuel Type'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing',
                                                                        'unestablished','unspecified','na','not-available','-1'] else x)

    data['Cylinders'] = data['Cylinders'].astype(float)
    data['Engine Size(L)'] = data['Engine Size(L)'].astype(float)

    data.drop(['Fuel Consumption City units','Fuel Consumption Hwy units','Fuel Consumption Comb units'], axis=1, inplace=True)
    data = data.rename(columns={'CO2 Emissions(g/km)':'CO2_Emissions', 'Engine Size(L)':'Engine_Size', 'Fuel Consumption City':'Fuel_Consumption_City', 
                                'Fuel Consumption Hwy':'Fuel_Consumption_Hwy', 'Fuel Consumption Comb':'Fuel_Consumption_Comb', 
                                'Fuel Type':'Fuel_Type', 'Vehicle Class':'Vehicle_Class'})

    return data

In [8]:
data_clean = data_preprocessing(df)
data_clean.head()

,Id,Make,Vehicle_Class,Engine_Size,Cylinders,Transmission,Fuel_Type,Fuel_Consumption_City,Fuel_Consumption_Hwy,Fuel_Consumption_Comb,CO2_Emissions
0,1,FOLD,PICKUP TRUCK - STANDARD,3.5,6.0,A6,X,8.640000,6.14,7.514791,306.0
1,2,CHEVO,PICKUP TRUCK - STANDARD,5.3,NaN,A6,E,27.270000,30.76,28.840000,283.0
2,3,BMV,SUBCOMPACT,4.4,NaN,M6,Z,1.700000,2.03,NaN,329.0
3,4,KIO,SUV - SMALL,NaN,4.0,AS6,X,15.337423,15.90,NaN,270.0
4,5,BARUSU,MINICOMPACT,3.0,6.0,NaN,Z,20.150000,6.00,13.800000,193.0


In [9]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137141 entries, 0 to 137140
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Id                     137141 non-null  int64  
 1   Make                   137141 non-null  object 
 2   Vehicle_Class          132009 non-null  object 
 3   Engine_Size            112808 non-null  float64
 4   Cylinders              116536 non-null  float64
 5   Transmission           126701 non-null  object 
 6   Fuel_Type              129722 non-null  object 
 7   Fuel_Consumption_City  131093 non-null  float64
 8   Fuel_Consumption_Hwy   130653 non-null  float64
 9   Fuel_Consumption_Comb  130651 non-null  float64
 10  CO2_Emissions          137141 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 11.5+ MB


In [10]:
data_clean.isna().sum()

Id                           0
Make                         0
Vehicle_Class             5132
Engine_Size              24333
Cylinders                20605
Transmission             10440
Fuel_Type                 7419
Fuel_Consumption_City     6048
Fuel_Consumption_Hwy      6488
Fuel_Consumption_Comb     6490
CO2_Emissions                0
dtype: int64

In [11]:
test_clean = data_preprocessing(test)